## Nauashev Alfarabi

In [1]:
import pandas as pd
import json
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('data.csv')

### tot_claim_cnt_l180d

**Description:** Sum of exposue of loans without TBC loans. Exposure means here field ""loan_summa"".

**Source:** contracts

**Key fields:** bank, loan_summa, contract_date

**Special notes:** 
1. Consider only loans where field ""bank"" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].
2. Disbursed loans means loans where contract_date is not null

***

The reason why code showing 0, because Today is **2024-10-27**. The max *claim_date* date in *data.csv* is **2024-02-13**. The differense between them more than 180 days.


I take taked the **2024-02-28** for example.

In [154]:
county = 0
max_claim_date = None

In [161]:
for index, row in df.iterrows():
    id = int(row['id'])
    
    if pd.notna(row['contracts']):
        contracts = json.loads(row['contracts'])
        if isinstance(contracts, list):
            filtered_contracts = [
                contract for contract in contracts 
                if isinstance(contract, dict) and 'claim_date' in contract and contract['claim_date']
            ]

            for contract in filtered_contracts:
                contract['claim_date'] = datetime.strptime(contract['claim_date'], "%d.%m.%Y")

            date_180_days_ago = datetime(2024, 2, 28) - timedelta(days=180)
            
            recent_claims_count = sum(
                1 for contract in filtered_contracts 
                if contract['claim_date'] >= date_180_days_ago
            )

            max_claim_date = max(
                max_claim_date, 
                max(contract['claim_date'] for contract in filtered_contracts)
            ) if max_claim_date else max(contract['claim_date'] for contract in filtered_contracts)
    
    if recent_claims_count > 0:
        print(f"ID: {id}\tNumber of claims in the last 180 days: {recent_claims_count}.")

ID: 2925211	Number of claims in the last 180 days: 58.
ID: 2925212	Number of claims in the last 180 days: 58.
ID: 2925213	Number of claims in the last 180 days: 58.
ID: 2925214	Number of claims in the last 180 days: 58.
ID: 2925215	Number of claims in the last 180 days: 58.
ID: 2925216	Number of claims in the last 180 days: 58.
ID: 2925217	Number of claims in the last 180 days: 58.
ID: 2925218	Number of claims in the last 180 days: 13.
ID: 2925219	Number of claims in the last 180 days: 13.
ID: 2925220	Number of claims in the last 180 days: 13.
ID: 2925221	Number of claims in the last 180 days: 13.
ID: 2925222	Number of claims in the last 180 days: 13.
ID: 2925223	Number of claims in the last 180 days: 13.
ID: 2925224	Number of claims in the last 180 days: 13.
ID: 2925225	Number of claims in the last 180 days: 13.
ID: 2925226	Number of claims in the last 180 days: 13.
ID: 2925227	Number of claims in the last 180 days: 13.
ID: 2925228	Number of claims in the last 180 days: 13.
ID: 292522

### disb_bank_loan_wo_tbc

**Description:** Sum of exposue of loans without TBC loans. Exposure means here field ""loan_summa"".

**Source:** contracts

**Key fields:** bank, loan_summa, contract_date

**Special notes:** 
1. Consider only loans where field ""bank"" is not in ['LIZ', 'LOM', 'MKO', 'SUG', null].
2. Disbursed loans means loans where contract_date is not null

***

In [8]:
loan_summa = 0
required_columns = ['bank', 'loan_summa', 'contract_date']

In [138]:
for index, row in df.iterrows():
    loan_summa = 0
    id = int(row['id'])
    
    if pd.notna(row['contracts']):
        contracts = json.loads(row['contracts'])
        if isinstance(contracts, list):
            for contract in contracts:
                if all(column in contract for column in required_columns):
                    if 'bank' in contract and contract['bank'].upper() not in ['LIZ', 'LOM', 'MKO', 'SUG']:
                        if contract['loan_summa']:
                            loan_summa += int(contract['loan_summa'])
    if loan_summa > 0:
        print(f'ID: {id}\tSum of loan: {loan_summa}')
    

ID: 2925231	Sum of loan: 2537851751
ID: 2925232	Sum of loan: 2537851751
ID: 2925243	Sum of loan: 559300000
ID: 2925244	Sum of loan: 559300000
ID: 2925250	Sum of loan: 2176827289
ID: 2925252	Sum of loan: 2176827289
ID: 2925254	Sum of loan: 2176827289
ID: 2925256	Sum of loan: 2176827289
ID: 2925276	Sum of loan: 9905106700
ID: 2925277	Sum of loan: 9905106700
ID: 2925279	Sum of loan: 1421662913
ID: 2925281	Sum of loan: 1421662913
ID: 2925283	Sum of loan: 1421662913
ID: 2925284	Sum of loan: 1421662913
ID: 2925291	Sum of loan: 233087600
ID: 2925293	Sum of loan: 233087600
ID: 2925295	Sum of loan: 233087600
ID: 2925298	Sum of loan: 300000000
ID: 2925300	Sum of loan: 233087600
ID: 2925305	Sum of loan: 300000000
ID: 2925312	Sum of loan: 300000000
ID: 2925316	Sum of loan: 300000000
ID: 2925384	Sum of loan: 10308484000
ID: 2925385	Sum of loan: 10308484000
ID: 2925388	Sum of loan: 3225259367
ID: 2925392	Sum of loan: 3225259367
ID: 2925396	Sum of loan: 3225259367
ID: 2925398	Sum of loan: 3225259367


### day_sinlastloan

**Description:** Number of days since last loan. 

**Source:** contracts

**Key fields:** contract_date, summa

**Special notes:**
1. Take last loan of client where summa is not null and calculate number of days from contract_date of this loan to application date.

***

In [129]:
application_date = None
required_columns = ['summa', 'contract_date']

In [176]:
for index, row in df.iterrows():
    max_contract_date = datetime.min
    days_num = 0
    id = int(row['id'])
    
    try:
        application_date = datetime.strptime(row['application_date'], "%Y-%m-%d %H:%M:%S.%f%z")
    except:
        application_date = datetime.strptime(row['application_date'], "%Y-%m-%d %H:%M:%S%z")
    
    application_date = application_date.replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=None)
        
    if pd.notna(row['contracts']):
        contracts = json.loads(row['contracts'])
        if isinstance(contracts, list):
            for contract in contracts:
                # print(contract)    
                if all(column in contract for column in required_columns) and contract['contract_date'] != '':
                        max_contract_date = max(max_contract_date, datetime.strptime(contract['contract_date'], "%d.%m.%Y"))
                
    days_num = (application_date - max_contract_date).days
    print(f'ID: {id}\tNumber of days from contract_date to application date: {days_num}')


ID: 2925210	Number of days from contract_date to application date: 738927
ID: 2925211	Number of days from contract_date to application date: 427
ID: 2925212	Number of days from contract_date to application date: 738927
ID: 2925213	Number of days from contract_date to application date: 427
ID: 2925214	Number of days from contract_date to application date: 738927
ID: 2925215	Number of days from contract_date to application date: 427
ID: 2925216	Number of days from contract_date to application date: 427
ID: 2925217	Number of days from contract_date to application date: 738927
ID: 2925218	Number of days from contract_date to application date: 288
ID: 2925219	Number of days from contract_date to application date: 738927
ID: 2925220	Number of days from contract_date to application date: 288
ID: 2925221	Number of days from contract_date to application date: 738927
ID: 2925222	Number of days from contract_date to application date: 288
ID: 2925223	Number of days from contract_date to applicatio